<a href="https://colab.research.google.com/github/1900690/yolov10/blob/main/yolov10-traking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

In [ ]:
!pip install -q supervision
!pip install -q git+https://github.com/THU-MIG/yolov10.git
!pip install deep-sort-realtime

In [ ]:
!mkdir -p weights
!wget -P weights -q https://github.com/jameslahm/yolov10/releases/download/v1.0/yolov10n.pt
!wget -P weights -q https://github.com/jameslahm/yolov10/releases/download/v1.0/yolov10s.pt
!wget -P weights -q https://github.com/jameslahm/yolov10/releases/download/v1.0/yolov10m.pt
!wget -P weights -q https://github.com/jameslahm/yolov10/releases/download/v1.0/yolov10b.pt
!wget -P weights -q https://github.com/jameslahm/yolov10/releases/download/v1.0/yolov10x.pt
!wget -P weights -q https://github.com/jameslahm/yolov10/releases/download/v1.0/yolov10l.pt
!ls -lh weights

In [ ]:
from IPython.display import HTML
from base64 import b64encode
def play_video(filename):
  html = ''
  video = open(filename,'rb').read()
  src = 'data:video/mp4;base64,' + b64encode(video).decode()
  html += fr'<video width=900 controls autoplay loop><source src="%s" type="video/mp4"></video>' % src
  return HTML(html)

In [ ]:
import cv2
import torch
import datetime
import numpy as np
from absl import app, flags
from absl.flags import FLAGS
from ultralytics import YOLOv10
from deep_sort_realtime.deepsort_tracker import DeepSort

In [ ]:
model_file = "/content/weights/yolov10x.pt"
video_file = "/content/drive/MyDrive/Colab Notebooks/test.mp4"
output_file = "/content/output.mp4"
conf = 0.5
class_id = 0
blur_id = None

# Initialize the video capture and the video writer objects
video_cap = cv2.VideoCapture(video_file)
frame_width = int(video_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(video_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(video_cap.get(cv2.CAP_PROP_FPS))

# Initialize the video writer object
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
writer = cv2.VideoWriter(output_file, fourcc, fps, (frame_width, frame_height))

# Initialize the DeepSort tracker
tracker = DeepSort(max_age=20, n_init=3)

# Check if GPU is available, otherwise use CPU
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print(f'Using device is {device}')

# Load the YOLO model
model = YOLOv10(model_file)
# model = models.get(FLAGS.model, pretrained_weights="coco").to(device)

# Load the COCO class labels the YOLO model was trained on
classes_path = "/content/coco.names"
with open(classes_path, "r") as f:
    class_names = f.read().strip().split("\n")

# Create a list of random colors to represent each class
np.random.seed(42)  # to get the same colors
colors = np.random.randint(0, 255, size=(len(class_names), 3))  # (80, 3)

while True:
    # Start time to compute the FPS
    start = datetime.datetime.now()

    # Read a frame from the video
    ret, frame = video_cap.read()

    # If there is no frame, we have reached the end of the video
    if not ret:
        print("End of the video file...")
        break

    # Perform object detection using the YOLO model on the current frame
    results = model(frame, verbose=False)[0]
    detect = []
    for det in results.boxes:
        confidence = det.conf
        label = det.cls
        bbox = det.xyxy[0]
        x1, y1, x2, y2 = map(int,bbox)
        class_id = int(label)

        # Filter out weak detections by confidence threshold and class_id
        if class_id is None:
            if confidence < conf:
                continue
        else:
            if class_id != class_id or confidence < conf:
                continue

        detect.append([[x1, y1, x2 - x1, y2 - y1], confidence, class_id])

    tracks = tracker.update_tracks(detect, frame=frame)

    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        ltrb = track.to_ltrb()
        class_id = track.get_det_class()
        x1, y1, x2, y2 = map(int, ltrb)
        color = colors[class_id]
        B, G, R = map(int, color)
        text = f"{track_id} - {class_names[class_id]}"

        cv2.rectangle(frame, (x1, y1), (x2, y2), (B, G, R), 2)
        cv2.rectangle(frame, (x1 - 1, y1 - 20), (x1 + len(text) * 12, y1), (B, G, R), -1)
        cv2.putText(frame, text, (x1 + 5, y1 - 8), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

        # Apply Gaussian Blur
        if blur_id is not None and class_id == blur_id:
            if 0 <= x1 < x2 <= frame.shape[1] and 0 <= y1 < y2 <= frame.shape[0]:
                frame[y1:y2, x1:x2] = cv2.GaussianBlur(frame[y1:y2, x1:x2], (99, 99), 3)

    # cv2.imshow('YOLOv9 Object tracking', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    # End time to compute the FPS
    end = datetime.datetime.now()

    # Show the time it took to process 1 frame
    print(f"Time to process 1 frame: {(end - start).total_seconds():.2f} seconds")

    # Calculate the frames per second and draw it on the frame
    fps = f"FPS: {1 / (end - start).total_seconds():.2f}"

    cv2.putText(frame, fps, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 8)
    writer.write(frame)

# Release video capture and writer
video_cap.release()
writer.release()

In [ ]:
!ffmpeg -hide_banner -loglevel error -i /content/output.mp4 -vcodec libx264 /content/output_test.mp4 -y
play_video("/content/output_test.mp4")

# Using Bytetrack

In [ ]:
!yolo tracker="bytetrack.yaml" mode=track model='/content/weights/yolov10s.pt' conf=0.25 source="/content/drive/MyDrive/Colab Notebooks/test.mp4" save=True show=True verbose=False save_crop